<a href="https://colab.research.google.com/github/hsuanchengsun/Natural-Language-Processing/blob/main/Word_Embedding_and_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word embeddings and sentiment analysis



## Section 0: Imports and Other Setup

I like to keep my imports and setup at the top of the file, so if I come back to the lab after a runtime disconnect I can just run one or two cells and know I have my Python environment setup properly.

In [ ]:
# The imports needed for this notebook
import numpy as np
import pandas as pd

In [ ]:
# Setup access to my Google Drive
from google.colab import drive
drive.mount('/content/drive',  force_remount=False)

Mounted at /content/drive


## Section 1: Create the Cleaned Reviews

Create a cleaned review (as a string) column with lemmas for each row of the data provided.

Similar to Step 6 in Lab Assignment 2.

### Read the CSV File

In [ ]:
# read a local csv file into Pandas DataFrame
%cd drive/MyDrive/Colab Notebooks/Mydata/
col_name = ['Cust_Rating', 'Datetime', 'Review', 'Resturant', 'City', 'State', 'Zipcode', 'Business_Rating_Score']
df = pd.read_csv("Restaurant_reviews_R.csv", header=None, names = col_name)
df.head(3)

/content/drive/MyDrive/Colab Notebooks/Mydata


,Cust_Rating,Datetime,Review,Resturant,City,State,Zipcode,Business_Rating_Score
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5


### Do the Contract Expansion

In [ ]:
# Contraction expansion
!pip install contractions
import contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 8.2 MB/s 
     |████████████████████████████████| 287 kB 64.7 MB/s 


In [ ]:
# Define contraction funtion
def contraction_expansion(text):
  expanded_word = []    
  for word in text.split():
  # using contractions.fix to expand
    expanded_word.append(contractions.fix(word))   
   
  return  ' '.join(expanded_word)

In [ ]:
# We start to work on contraction 
df['Review_expanded'] = df['Review'].apply(contraction_expansion)
df.head()

,Cust_Rating,Datetime,Review,Resturant,City,State,Zipcode,Business_Rating_Score,Review_expanded
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ..."
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,Always a great place to taste some tea with fr...
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,Delish! Stopped in on a random Sunday and so g...


In [ ]:
# tokenize each review in the dataframe
from nltk.tokenize import wordpunct_tokenize
%time df['Review_Token'] = df['Review_expanded'].apply(wordpunct_tokenize)
df.head()

CPU times: user 1.94 s, sys: 298 ms, total: 2.24 s
Wall time: 2.25 s


,Cust_Rating,Datetime,Review,Resturant,City,State,Zipcode,Business_Rating_Score,Review_expanded,Review_Token
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...,"[its, the, best, pizza, hut, i, ', v, found, i..."
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ...","[This, place, is, an, interesting, combo, ., T..."
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...,"[Pizza, Hut, is, great, !, You, get, huge, piz..."
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,Always a great place to taste some tea with fr...,"[Always, a, great, place, to, taste, some, tea..."
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,Delish! Stopped in on a random Sunday and so g...,"[Delish, !, Stopped, in, on, a, random, Sunday..."


### Create POS tags

In [ ]:
# import POS libiaries
import nltk
nltk.download('punkt') 
nltk.download('averaged_perceptron_tagger') 
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tag import pos_tag

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# create POS tag
%time df['Review_tagged'] = df['Review_Token'].apply(nltk.pos_tag)
df.head()

CPU times: user 4min 59s, sys: 2.55 s, total: 5min 1s
Wall time: 5min 2s


,Cust_Rating,Datetime,Review,Resturant,City,State,Zipcode,Business_Rating_Score,Review_expanded,Review_Token,Review_tagged
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...,"[its, the, best, pizza, hut, i, ', v, found, i...","[(its, PRP$), (the, DT), (best, JJS), (pizza, ..."
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ...","[This, place, is, an, interesting, combo, ., T...","[(This, DT), (place, NN), (is, VBZ), (an, DT),..."
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...,"[Pizza, Hut, is, great, !, You, get, huge, piz...","[(Pizza, NNP), (Hut, NNP), (is, VBZ), (great, ..."
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,Always a great place to taste some tea with fr...,"[Always, a, great, place, to, taste, some, tea...","[(Always, VBZ), (a, DT), (great, JJ), (place, ..."
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,Delish! Stopped in on a random Sunday and so g...,"[Delish, !, Stopped, in, on, a, random, Sunday...","[(Delish, JJ), (!, .), (Stopped, VBN), (in, IN..."


### Lemmatization

In [ ]:
# create a function to lemmatize words based on different POS tag types; 
# slightly change from earlier as input is not a tagged corpus, but an individual tagged column
from nltk.stem import WordNetLemmatizer
def lemmatize_POS (tagged_columns):
  wnl = WordNetLemmatizer()
  text=[]
  for word, tag in tagged_columns:
    if tag.startswith('NN'):
      token = wnl.lemmatize(word, pos='n') # n is NOUN 
    elif tag.startswith('VB'):
      token = wnl.lemmatize(word, pos='v') # v is VERB
    elif tag.startswith('JJ'):
      token = wnl.lemmatize(word, pos='a') # a is ADJ
    elif tag.startswith('RB'):
      token = wnl.lemmatize(word, pos='r') # r is ADV
    else:
      token = word
    text.append(token)
  #cleaned_corpus.append(text)
  return text

In [ ]:
%time df['Review_lemmas'] = df['Review_tagged'].apply(lemmatize_POS)
df['Review_lemmas'].head()

CPU times: user 21.6 s, sys: 237 ms, total: 21.8 s
Wall time: 21.9 s


0    [its, the, best, pizza, hut, i, ', v, find, in...
1    [This, place, be, an, interesting, combo, ., T...
2    [Pizza, Hut, be, great, !, You, get, huge, piz...
3    [Always, a, great, place, to, taste, some, tea...
4    [Delish, !, Stopped, in, on, a, random, Sunday...
Name: Review_lemmas, dtype: object

### Text normalization

In [ ]:
# Define preprocessing function
from nltk.corpus import stopwords
nltk.download('stopwords')
from string import punctuation

def preprocessing(tokens):
  tokens = [token.lower() for token in tokens] # lowercasing
  
  tokens = [token for token in tokens if not token.isdigit()] # remove digit

  tokens = [token for token in tokens if token not in punctuation] # remove punctuations

  mystopwords = set(stopwords.words("english")) 
  tokens = [token for token in tokens if token not in mystopwords] # remove stopwords
  
  tokens = [word for word in tokens if len(word)>=3] # remove words with one or two characters

  return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# now let's apply text normalization
%time df['Review_lemmas_final'] = df['Review_lemmas'].apply(preprocessing)
df['Review_lemmas_final'].head()

CPU times: user 11.8 s, sys: 1.12 s, total: 12.9 s
Wall time: 12.9 s


0    [best, pizza, hut, find, florida, always, get,...
1    [place, interesting, combo, chef, david, incre...
2    [pizza, hut, great, get, huge, pizza, money, s...
3    [always, great, place, taste, tea, friend, sta...
4    [delish, stopped, random, sunday, glad, normal...
Name: Review_lemmas_final, dtype: object

### Remove rows if list of lemmas is empty

In [ ]:
# create a text column with tagged tokens
%time df['Cleaned_review_lemma'] = df['Review_lemmas_final'].apply(lambda row: ' '.join(str(x) for x in row))
df.head()

CPU times: user 648 ms, sys: 18.1 ms, total: 666 ms
Wall time: 669 ms


,Cust_Rating,Datetime,Review,Resturant,City,State,Zipcode,Business_Rating_Score,Review_expanded,Review_Token,Review_tagged,Review_lemmas,Review_lemmas_final,Cleaned_review_lemma
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...,"[its, the, best, pizza, hut, i, ', v, found, i...","[(its, PRP$), (the, DT), (best, JJS), (pizza, ...","[its, the, best, pizza, hut, i, ', v, find, in...","[best, pizza, hut, find, florida, always, get,...",best pizza hut find florida always get great d...
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ...","[This, place, is, an, interesting, combo, ., T...","[(This, DT), (place, NN), (is, VBZ), (an, DT),...","[This, place, be, an, interesting, combo, ., T...","[place, interesting, combo, chef, david, incre...",place interesting combo chef david incredible ...
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...,"[Pizza, Hut, is, great, !, You, get, huge, piz...","[(Pizza, NNP), (Hut, NNP), (is, VBZ), (great, ...","[Pizza, Hut, be, great, !, You, get, huge, piz...","[pizza, hut, great, get, huge, pizza, money, s...",pizza hut great get huge pizza money service q...
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,Always a great place to taste some tea with fr...,"[Always, a, great, place, to, taste, some, tea...","[(Always, VBZ), (a, DT), (great, JJ), (place, ...","[Always, a, great, place, to, taste, some, tea...","[always, great, place, taste, tea, friend, sta...",always great place taste tea friend staff frie...
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,Delish! Stopped in on a random Sunday and so g...,"[Delish, !, Stopped, in, on, a, random, Sunday...","[(Delish, JJ), (!, .), (Stopped, VBN), (in, IN...","[Delish, !, Stopped, in, on, a, random, Sunday...","[delish, stopped, random, sunday, glad, normal...",delish stopped random sunday glad normally eat...


In [ ]:
# Test to see if there was any empty reviews in the dataset
df['Review_lemmas_final'].isnull().values.any()

False

### Section 1 Answer

Show the final number of rows in the cleaned dataframe

In [ ]:
# I like to save my result to a CSV file here, so that if I get disconnecteded
# or have to come back to the work later, I don't waste time waiting for the POS
# tagging to complete to recreate my dataframe.

In [ ]:
# final number of rows in the cleaned dataframe
print("final number of rows in the cleaned dataframe: ", len(df))

final number of rows in the cleaned dataframe:  63230


In [ ]:
# Save the dataframe into our drive, so we can load it later for analysis
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Mydata/restaurant_review_row.csv')

## Section 2: Create Dataframe with FastText Embeddings as Features

### Load the Fasttext Pre-trained Model

The fasttext-wiki-news-subwords-300 is about 960MB and can be downloaded from: (https://drive.google.com/file/d/1E4Ag_QqobIAGZZaUeqAQxnw2CyCfNskK/view).

Download the file and save in under '/content/drive/MyDrive/Mydata/fasttext-wiki-news-subwords-300.gz'. You should save it under your own google drive folder, and use the left file navigation to copy the file path.

In [ ]:
# Import Libararies

import pickle

import warnings #This module ignores the various types of warnings generated
import os

warnings.filterwarnings("ignore")

import psutil #This module helps in retrieving information on running processes and system resource utilization
process = psutil.Process(os.getpid())
from psutil import virtual_memory
mem = virtual_memory()

from gensim.models import Word2Vec, KeyedVectors

In [ ]:
# The imports needed for this notebook
import numpy as np
import pandas as pd

In [ ]:
# Setup access to my Google Drive
from google.colab import drive
drive.mount('/content/drive',  force_remount=False)

Mounted at /content/drive


In [ ]:
# define the path to read the pre-trained vectors
pretrainedpath = '/content/drive/MyDrive/Colab Notebooks/Mydata/fasttext-wiki-news-subwords-300.gz'

In [ ]:
#Load Word2Vect model. This will take some time, but it is a one time effort! 
pre = process.memory_info().rss
print("Memory used in GB before Loading the Model: %0.2f"%float(pre/(10**9))) #Check memory usage before loading the model
print('-'*10)

import time #This module is used to calculate the time 
start_time = time.time() #Start the timer
ttl = mem.total #Toal memory available

# load the model using the pretrained path we defined earlier. 
#w2v_model = KeyedVectors.load_word2vec_format(pretrainedpath, binary=True) 
w2v_model = KeyedVectors.load_word2vec_format(pretrainedpath) 

print("%0.2f seconds taken to load"%float(time.time() - start_time)) #Calculate the total time elapsed since starting the timer
print('-'*10)

print('Finished loading Word2Vec')
print('-'*10)

post = process.memory_info().rss
print("Memory used in GB after Loading the Model: {:.2f}".format(float(post/(10**9)))) #Calculate the memory used after loading the model
print('-'*10)

print("Percentage increase in memory usage: {:.2f}% ".format(float((post/pre)*100))) #Percentage increase in memory after loading the model
print('-'*10)

print("Number of words in vocablulary: ",len(w2v_model.vocab)) #Number of words in the vocabulary. 

Memory used in GB before Loading the Model: 0.19
----------
198.46 seconds taken to load
----------
Finished loading Word2Vec
----------
Memory used in GB after Loading the Model: 1.81
----------
Percentage increase in memory usage: 929.53% 
----------
Number of words in vocablulary:  999999


### Create the 1% Token List

See Lab L3-TF-IDF Section 4.2

In [ ]:
# read a local csv file into Pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Mydata/restaurant_review_row.csv')
df.head(3)

,Unnamed: 0,Cust_Rating,Datetime,Review,Resturant,City,State,Zipcode,Business_Rating_Score,Review_expanded,Review_Token,Review_tagged,Review_lemmas,Review_lemmas_final,Cleaned_review_lemma
0,0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...,"['its', 'the', 'best', 'pizza', 'hut', 'i', ""'...","[('its', 'PRP$'), ('the', 'DT'), ('best', 'JJS...","['its', 'the', 'best', 'pizza', 'hut', 'i', ""'...","['best', 'pizza', 'hut', 'find', 'florida', 'a...",best pizza hut find florida always get great d...
1,1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ...","['This', 'place', 'is', 'an', 'interesting', '...","[('This', 'DT'), ('place', 'NN'), ('is', 'VBZ'...","['This', 'place', 'be', 'an', 'interesting', '...","['place', 'interesting', 'combo', 'chef', 'dav...",place interesting combo chef david incredible ...
2,2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...,"['Pizza', 'Hut', 'is', 'great', '!', 'You', 'g...","[('Pizza', 'NNP'), ('Hut', 'NNP'), ('is', 'VBZ...","['Pizza', 'Hut', 'be', 'great', '!', 'You', 'g...","['pizza', 'hut', 'great', 'get', 'huge', 'pizz...",pizza hut great get huge pizza money service q...


In [ ]:
# create our list of lists corpus (a list of lists of tokens) 
reviews = df['Cleaned_review_lemma'].tolist()

print("The data type of reviews:", type(reviews))
print("The data type of the first review:", type(reviews[0]))

reviews[0]

The data type of reviews: <class 'list'>
The data type of the first review: <class 'str'>


'best pizza hut find florida always get great deal offer staff friendly make feel welcome child make feel extra special welcome huge way'

In [ ]:
# included words used by at least 1% of the documents as features
print("The number of reviews:", len(reviews))
print("1% of the documents: ", round(len(reviews) * 0.01))

The number of reviews: 63230
1% of the documents:  632


In [ ]:
import gensim
from gensim import corpora

# create our list of lists corpus (a list of lists of tokens) 
corpus = [str(doc).split() for doc in reviews]

print(*corpus[0])

best pizza hut find florida always get great deal offer staff friendly make feel welcome child make feel extra special welcome huge way


In [ ]:
# create a dictionary of tokens
dictionary = gensim.corpora.Dictionary(corpus)

In [ ]:
# create a token list with a minimal document frequency count (number of documents which a token occurs)
token_list =[]
for token_id, count in dictionary.dfs.items(): # .dfs returns doctument frequecy based on token id (int)
  if count>632:
    token_list.append(dictionary.get(token_id)) #.get() retrieve the actual tokens based on id

print(len(token_list))
print(token_list[:3]) # see the list of tokens 

777
['best', 'pizza', 'find']


### Use gensim.corpora.Dictionary

The dfs object will count the number of times a term appears in a document. (The documentation: https://radimrehurek.com/gensim/corpora/dictionary.html)

In [ ]:
# create a vector list for Top 1% tokens
vector_list = [w2v_model[token] for token in token_list if token in w2v_model.vocab]
len(vector_list)

775

In [ ]:
# similarly, create a word list 
words_filtered = [token for token in token_list if token in w2v_model.vocab]
len(words_filtered)

775

In [ ]:
# Zip the words together with their vector representations
word_vec_zip = zip(words_filtered, vector_list)

### Section 2 Part C Answer

Output a dataframe snipet that shows a few of the top 1% words and their associated vectors. (see output in Lab L3-Word Embeddings Part II Section 4.2)

In [ ]:
# Cast to a dict and then turn it into a DataFrame
word_vec_dict = dict(word_vec_zip)
df_word = pd.DataFrame.from_dict(word_vec_dict, orient='index')
df_word.head(5)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
best,0.001844,0.024138,-0.015031,0.029335,0.011248,-0.086815,-0.014381,-0.12338,-0.110670,-0.023453,...,0.011360,-0.039884,0.003739,-0.105800,0.038353,-0.019798,0.025037,0.012728,-0.026845,-0.036220
pizza,-0.048900,-0.007654,-0.063315,-0.027614,0.021633,-0.066629,-0.021372,-0.09437,0.092511,0.051828,...,-0.028695,-0.031919,-0.033987,-0.067878,0.065588,-0.095538,0.036694,0.126910,-0.013699,-0.042434
find,0.046331,0.021415,0.021743,-0.012192,-0.087447,0.012167,0.060469,-0.12495,-0.068757,0.030743,...,-0.024371,0.012181,-0.023591,-0.025709,0.019614,-0.030628,0.059684,-0.021438,-0.036149,0.004188
always,0.019443,0.012101,0.036422,-0.024460,-0.040211,0.001140,0.005575,-0.10744,0.005194,-0.054232,...,-0.009170,0.026573,0.041074,-0.014170,0.020027,0.001608,0.010946,0.024232,-0.016167,-0.018401
get,-0.009659,0.095145,0.043168,-0.019902,-0.035841,-0.032168,-0.006862,-0.11612,-0.009533,-0.025850,...,-0.013236,-0.024453,-0.118840,-0.013272,0.015117,-0.018286,-0.066435,0.019069,-0.032215,-0.065898


### Create a Dataframe using word2vec vectors as features

See Lab L3-Word Embeddings Part II Section 4.3.

Note that, after you remove all non-vocabulary words from review_cleaned, you might end up with an empty review. If so you will need to address that because the Word2Vec model will not process an empty list.

In [ ]:
df.rename(columns = {"Unnamed: 0": "reviewID"}, inplace = True)
df.head()

,reviewID,Cust_Rating,Datetime,Review,Resturant,City,State,Zipcode,Business_Rating_Score,Review_expanded,Review_Token,Review_tagged,Review_lemmas,Review_lemmas_final,Cleaned_review_lemma
0,0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...,"['its', 'the', 'best', 'pizza', 'hut', 'i', ""'...","[('its', 'PRP$'), ('the', 'DT'), ('best', 'JJS...","['its', 'the', 'best', 'pizza', 'hut', 'i', ""'...","['best', 'pizza', 'hut', 'find', 'florida', 'a...",best pizza hut find florida always get great d...
1,1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ...","['This', 'place', 'is', 'an', 'interesting', '...","[('This', 'DT'), ('place', 'NN'), ('is', 'VBZ'...","['This', 'place', 'be', 'an', 'interesting', '...","['place', 'interesting', 'combo', 'chef', 'dav...",place interesting combo chef david incredible ...
2,2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...,"['Pizza', 'Hut', 'is', 'great', '!', 'You', 'g...","[('Pizza', 'NNP'), ('Hut', 'NNP'), ('is', 'VBZ...","['Pizza', 'Hut', 'be', 'great', '!', 'You', 'g...","['pizza', 'hut', 'great', 'get', 'huge', 'pizz...",pizza hut great get huge pizza money service q...
3,3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,Always a great place to taste some tea with fr...,"['Always', 'a', 'great', 'place', 'to', 'taste...","[('Always', 'VBZ'), ('a', 'DT'), ('great', 'JJ...","['Always', 'a', 'great', 'place', 'to', 'taste...","['always', 'great', 'place', 'taste', 'tea', '...",always great place taste tea friend staff frie...
4,4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,Delish! Stopped in on a random Sunday and so g...,"['Delish', '!', 'Stopped', 'in', 'on', 'a', 'r...","[('Delish', 'JJ'), ('!', '.'), ('Stopped', 'VB...","['Delish', '!', 'Stopped', 'in', 'on', 'a', 'r...","['delish', 'stopped', 'random', 'sunday', 'gla...",delish stopped random sunday glad normally eat...


In [ ]:
# create a function that simply calculates an average of all word vectors within a document
def document_vector(word2vec_model, doc):
  return np.mean(word2vec_model[doc], axis =0)

In [ ]:
# use df_bow['Review_Token'] column which is already cleaned
reviews = [[word.replace("'", '').strip() for word in review[1:-1].split(',')] for review in df['Review_Token'].tolist()]
reviews_clean = [[word for word in review if word in w2v_model.vocab] for review in reviews]

In [ ]:
#use the function to create an average of all word2vec vectors
review_vectors = []
for doc in reviews_clean:  # append the vector for each document
  if not doc:
    review_vectors.append(np.array([None] * 300)) # if doc is an empty review, add an Null vector to the list
  else: review_vectors.append(document_vector(w2v_model, doc))

review_vectors_a = np.array(review_vectors)  # list to array

In [ ]:
review_vectors_a[0]

array([0.000465957447886467, 0.0008393784519284964, 0.02626086212694645,
       0.006459311116486788, -0.023641329258680344,
       -0.0014703957131132483, -0.00340786250308156, -0.0794844850897789,
       -0.03397774323821068, -0.007223162800073624, -0.025051552802324295,
       -0.04717204347252846, 0.032542359083890915, -0.019905542954802513,
       0.009996919892728329, 0.003823333652690053, 0.10160589963197708,
       0.017124395817518234, 0.0799848884344101, 0.006466304883360863,
       -0.018795939162373543, 0.010199625045061111, -0.01765279844403267,
       0.07833908498287201, 0.006654043681919575, 0.00276313629001379,
       0.034834668040275574, -0.00033373755286447704, 0.04373334348201752,
       0.01665361598134041, -0.021969038993120193, -0.0008735370356589556,
       -0.007370221894234419, -0.013412836007773876, 0.010694407857954502,
       -0.028468692675232887, -0.010987120680510998,
       0.00037390636862255633, -0.01647082157433033,
       -0.0023401163052767515, 0.

In [ ]:
# convert our doc embedding vectors to dataframe
labels = ['AWE' + str(i) for i in range (1, 301)]
awe_df = pd.DataFrame(review_vectors_a, index=df['reviewID'].tolist(), columns=labels)
awe_df = awe_df.reset_index()
awe_df.rename(columns={'index':'reviewID'}, inplace=True)
awe_df.head()

,reviewID,AWE1,AWE2,AWE3,AWE4,AWE5,AWE6,AWE7,AWE8,AWE9,...,AWE291,AWE292,AWE293,AWE294,AWE295,AWE296,AWE297,AWE298,AWE299,AWE300
0,0,0.000466,0.000839,0.026261,0.006459,-0.023641,-0.00147,-0.003408,-0.079484,-0.033978,...,-0.005968,0.025351,-0.029017,0.000196,0.017939,-0.012416,0.011076,0.014994,0.000625,-0.00643
1,1,-0.010688,-0.001212,0.020719,0.003003,-0.007375,-0.020803,-0.000588,-0.096937,-0.016457,...,0.002474,0.011572,-0.015873,-0.002296,-0.001984,-0.005425,-0.004394,-0.010526,-0.005332,-0.009384
2,2,-0.005978,-0.000683,0.010486,0.001734,-0.023263,-0.010447,-0.017599,-0.069113,-0.018448,...,-0.000389,0.017479,-0.027804,-0.004635,0.007689,-0.005709,0.014056,0.007208,-0.013408,-0.009189
3,3,-0.006715,0.001068,0.020165,0.007803,-0.042985,-0.004677,-0.007698,-0.067961,-0.017233,...,-0.013978,0.025447,-0.008755,-0.001572,0.000808,-0.00436,0.009962,0.01973,-0.025982,0.002044
4,4,-0.01309,0.010157,0.010822,0.008238,0.001968,-0.008857,-0.010102,-0.070799,-0.021641,...,0.0034,0.015685,-0.023062,0.005128,0.005895,-0.000231,0.004912,-0.014134,-0.007396,-0.004058


In [ ]:
# merge join with the original dataset using reviewID 
review_AWE = pd.merge(left=df, right=awe_df, left_on='reviewID', right_on='reviewID')
review_AWE.head(3)

,reviewID,Cust_Rating,Datetime,Review,Resturant,City,State,Zipcode,Business_Rating_Score,Review_expanded,...,AWE291,AWE292,AWE293,AWE294,AWE295,AWE296,AWE297,AWE298,AWE299,AWE300
0,0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...,...,-0.005968,0.025351,-0.029017,0.000196,0.017939,-0.012416,0.011076,0.014994,0.000625,-0.00643
1,1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ...",...,0.002474,0.011572,-0.015873,-0.002296,-0.001984,-0.005425,-0.004394,-0.010526,-0.005332,-0.009384
2,2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...,...,-0.000389,0.017479,-0.027804,-0.004635,0.007689,-0.005709,0.014056,0.007208,-0.013408,-0.009189


In [ ]:
# Test to see if there was any empty reviews in the dataset
review_AWE.isnull().values.any()

True

In [ ]:
# Need to drop rows that have an empty review
review_AWE.dropna(inplace = True)
review_AWE.isnull().values.any()

False

### Section 2 Answer

Output a snipet of the first 5 rows of the dataframe that includes the metadata and the word2vec feature vector for each review.

In [ ]:
# Save your answer to section 2 in a CSV file, you will need it later
# for section 4 and you don't want to have to recreate it.

In [ ]:
# final number of rows in the cleaned dataframe
print("final number of rows in the cleaned dataframe: ", len(review_AWE))

final number of rows in the cleaned dataframe:  63216


In [ ]:
review_AWE.head()

,reviewID,Cust_Rating,Datetime,Review,Resturant,City,State,Zipcode,Business_Rating_Score,Review_expanded,...,AWE291,AWE292,AWE293,AWE294,AWE295,AWE296,AWE297,AWE298,AWE299,AWE300
0,0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...,...,-0.005968,0.025351,-0.029017,0.000196,0.017939,-0.012416,0.011076,0.014994,0.000625,-0.00643
1,1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ...",...,0.002474,0.011572,-0.015873,-0.002296,-0.001984,-0.005425,-0.004394,-0.010526,-0.005332,-0.009384
2,2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...,...,-0.000389,0.017479,-0.027804,-0.004635,0.007689,-0.005709,0.014056,0.007208,-0.013408,-0.009189
3,3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,Always a great place to taste some tea with fr...,...,-0.013978,0.025447,-0.008755,-0.001572,0.000808,-0.00436,0.009962,0.01973,-0.025982,0.002044
4,4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,Delish! Stopped in on a random Sunday and so g...,...,0.0034,0.015685,-0.023062,0.005128,0.005895,-0.000231,0.004912,-0.014134,-0.007396,-0.004058


In [ ]:
# Save the dataframe into our drive, so we can load it later for analysis
review_AWE.to_csv('/content/drive/MyDrive/Colab Notebooks/Mydata/restaurant_review_AWE.csv', index=False)

## Section 3: Train Custom Word2Vec Embedding

See Lab L3-Word Embeddings Part II, Section "Part 3: Learn embeddings from scratch"

### Create The Sentences

In [ ]:
# read a local csv file into Pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Mydata/restaurant_review_row.csv')
df.head(3)

,Unnamed: 0,Cust_Rating,Datetime,Review,Resturant,City,State,Zipcode,Business_Rating_Score,Review_expanded,Review_Token,Review_tagged,Review_lemmas,Review_lemmas_final,Cleaned_review_lemma
0,0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...,"['its', 'the', 'best', 'pizza', 'hut', 'i', ""'...","[('its', 'PRP$'), ('the', 'DT'), ('best', 'JJS...","['its', 'the', 'best', 'pizza', 'hut', 'i', ""'...","['best', 'pizza', 'hut', 'find', 'florida', 'a...",best pizza hut find florida always get great d...
1,1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ...","['This', 'place', 'is', 'an', 'interesting', '...","[('This', 'DT'), ('place', 'NN'), ('is', 'VBZ'...","['This', 'place', 'be', 'an', 'interesting', '...","['place', 'interesting', 'combo', 'chef', 'dav...",place interesting combo chef david incredible ...
2,2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...,"['Pizza', 'Hut', 'is', 'great', '!', 'You', 'g...","[('Pizza', 'NNP'), ('Hut', 'NNP'), ('is', 'VBZ...","['Pizza', 'Hut', 'be', 'great', '!', 'You', 'g...","['pizza', 'hut', 'great', 'get', 'huge', 'pizz...",pizza hut great get huge pizza money service q...


In [ ]:
# create our list of lists corpus (a list of lists of tokens) 
reviews = df['Cleaned_review_lemma'].tolist()

print("The data type of reviews:", type(reviews))
print("The data type of the first review:", type(reviews[0]))

reviews[0]

The data type of reviews: <class 'list'>
The data type of the first review: <class 'str'>


'best pizza hut find florida always get great deal offer staff friendly make feel welcome child make feel extra special welcome huge way'

In [ ]:
# create a stream of sentences (a list of list of words) from the original text
sent = [str(doc).split() for doc in reviews]
sent[0]

['best',
 'pizza',
 'hut',
 'find',
 'florida',
 'always',
 'get',
 'great',
 'deal',
 'offer',
 'staff',
 'friendly',
 'make',
 'feel',
 'welcome',
 'child',
 'make',
 'feel',
 'extra',
 'special',
 'welcome',
 'huge',
 'way']

### Train the Bigram Model

In [ ]:
from gensim.models.phrases import Phrases, Phraser

In [ ]:
# train a bigram model from the stream of sentences. We set min_count as 30 here. 
bigram = Phrases(sent, min_count=30)

In [ ]:
# explort the trained bigram model
# Use this instead of Phrases because we do not need to update the bigram statistics any more.
bigram_phraser = Phraser(bigram) 

In [ ]:
# apply the exported model to each sentence of the corpus
# sentences are ready for training a Word2Vec() instance
sentences = bigram_phraser[sent]

### Collect Frequent Words and Phrases

In [ ]:
# let's create a list of bigram phrases
bigrams = []
for sent in sentences:
  for word in sent: 
    if '_' in word:
      bigrams.append(word)

In [ ]:
# number of bigrams detected
len(bigrams)

170768

In [ ]:
from collections import defaultdict

In [ ]:
# let's check overall word frequency
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1

len(word_freq)

47699

In [ ]:
# sort the dictionary based on the values (frequency count) from high to low
sorted_word_freq = sorted(word_freq.items(), key=lambda kv: kv[1], reverse=True)

In [ ]:
# the top 20 used words
sorted_word_freq [:20]

[('food', 45212),
 ('good', 40114),
 ('place', 37312),
 ('get', 35225),
 ('order', 33959),
 ('great', 24368),
 ('like', 22674),
 ('one', 20801),
 ('would', 20643),
 ('service', 18528),
 ('come', 17775),
 ('time', 17263),
 ('try', 16959),
 ('make', 16034),
 ('really', 14939),
 ('say', 13793),
 ('restaurant', 13603),
 ('also', 13420),
 ('...', 13149),
 ('take', 12809)]

In [ ]:
# let's check overall phrases frequency
phrases_freq = defaultdict(int)
for big in bigrams:
  phrases_freq[big] += 1

len(phrases_freq)

1280

In [ ]:
# sort the dictionary based on the values (frequency count) from high to low
sorted_phrases_freq = sorted(phrases_freq.items(), key=lambda kv: kv[1], reverse=True)

In [ ]:
# the top 20 used phrases
sorted_phrases_freq[:20]

[('come_back', 3944),
 ('first_time', 2716),
 ('customer_service', 2563),
 ('highly_recommend', 1787),
 ('next_time', 1672),
 ('ice_cream', 1561),
 ('even_though', 1309),
 ('every_time', 1303),
 ('definitely_back', 1287),
 ('make_sure', 1286),
 ('feel_like', 1260),
 ('would_recommend', 1182),
 ('happy_hour', 1162),
 ('staff_friendly', 1141),
 ('cannot_wait', 945),
 ('mac_cheese', 896),
 ('last_night', 826),
 ('friendly_staff', 807),
 ('wait_minute', 800),
 ('drive_thru', 791)]

*Section 3 Part B Answer*

Output a list of the top twenty phrases and the top twenty words.

### Learn the Custom Embedding

In [ ]:
from gensim.models import Word2Vec, KeyedVectors

t = time.time()
# Use the default COBW model, which is faster to train.
w2v_model_custom = Word2Vec(sentences, 
                       min_count=30, window=7,
                       size=300, # we use 300 here to compare with pre-trained embeddings later. It is usually depending on your dataset size. 
                       workers = 2,
                       iter=20)
print('Time to train the model: {} seconds'.format(round(time.time() - t, 1)))

Time to train the model: 355.1 seconds


In [ ]:
# let's make our model read-only. 
w2v_model_custom.init_sims(replace=True)

### Compare learned and pre-trained embedding

*Section 3 Part D Answer*

Print out the 10 most similar words for the 2 most frequent terms in the corpus and include the similarity scores. (See Lab L3-Word Embeddings Part II Section 3.4 for an example.)

In [ ]:
# let's compare our custom embeddings vs. google news pre-trained embeddings
# bedside_manner and listen becomes a highly contextualized word
print("Our training models on food:\n")
pd.DataFrame(w2v_model_custom.most_similar(positive=['food']), columns = ['similar word', 'similarity score'])

Our training models on food:



,similar word,similarity score
0,service,0.419976
1,everything,0.349133
2,cuisine,0.342921
3,sushi,0.339789
4,meal,0.337463
5,panda_express,0.334606
6,chip_salsa,0.323704
7,restaurant,0.322981
8,value_money,0.313233
9,takeout,0.306092


In [ ]:
# let's compare our custom embeddings vs. google news pre-trained embeddings
# bedside_manner and listen becomes a highly contextualized word

print("Pre-trained model on food:\n")
pd.DataFrame(w2v_model.most_similar(positive=['food']), columns = ['similar word', 'similarity score'])

Pre-trained model on food:



,similar word,similarity score
0,foods,0.776650
1,food-,0.759973
2,foodstuff,0.735816
3,foodless,0.732013
4,food.,0.730203
5,foodstuffs,0.728274
6,healthfood,0.727759
7,dogfood,0.718351
8,catfood,0.716797
9,non-food,0.716789


In [ ]:
# let's compare our custom embeddings vs. google news pre-trained embeddings
# bedside_manner and listen becomes a highly contextualized word
print("Our training models on good:\n")
pd.DataFrame(w2v_model_custom.most_similar(positive=['good']), columns = ['similar word', 'similarity score'])

Our training models on good:



,similar word,similarity score
0,decent,0.634491
1,great,0.587786
2,alright,0.552228
3,tasty,0.498587
4,solid,0.479211
5,average,0.457959
6,wise,0.450796
7,nothing_special,0.443417
8,well,0.432828
9,bit_pricey,0.427541


In [ ]:
# let's compare our custom embeddings vs. google news pre-trained embeddings
# bedside_manner and listen becomes a highly contextualized word

print("Pre-trained model on food:\n")
pd.DataFrame(w2v_model.most_similar(positive=['good']), columns = ['similar word', 'similarity score'])

Pre-trained model on food:



,similar word,similarity score
0,bad,0.850309
1,goood,0.810768
2,excellent,0.799670
3,decent,0.781157
4,.good,0.764746
5,great,0.764050
6,nice,0.761053
7,not-so-good,0.754202
8,good--and,0.747264
9,good-enough,0.747109


*Section 3 Part E Answer*

Briefly discuss the differences you see between the output of the pre-trained model and the output of the custom embedding.

Compared to the pre-trained model, our training model on the "food" word shows lower similarity scores for the most similar 10 words. The most similar 10 words on "good" also show that the pre-trained model has higher similarity scores for each word. For the content of these words, there are more topic-related words in our model. For instance, "food" is related to service, cuisine, and meal, and "good" is similar to decent and tasty since our training data is from restaurant reviews. On the other hand, in the pre-trained model, the simliar words contain the words. For example, food is similar to foods, dogfood, and catfood, and good is related to goood and good-enough.

## Section 4 - Create document embedding features for aggregated reviews

After aggregation you should have 1075 rows. You can start with your answer to Section 2.

In [ ]:
# read a local csv file into Pandas DataFrame
df_AWE = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Mydata/restaurant_review_AWE.csv')
df_AWE.head(3)

,reviewID,Cust_Rating,Datetime,Review,Resturant,City,State,Zipcode,Business_Rating_Score,Review_expanded,...,AWE291,AWE292,AWE293,AWE294,AWE295,AWE296,AWE297,AWE298,AWE299,AWE300
0,0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...,...,-0.005968,0.025351,-0.029017,0.000196,0.017939,-0.012416,0.011076,0.014994,0.000625,-0.006430
1,1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ...",...,0.002474,0.011572,-0.015873,-0.002296,-0.001984,-0.005425,-0.004394,-0.010526,-0.005332,-0.009384
2,2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...,...,-0.000389,0.017479,-0.027804,-0.004635,0.007689,-0.005709,0.014056,0.007208,-0.013408,-0.009189


### Aggregate by individual restaurant and calculate the mean document embedding.

In [ ]:
# Get the metadata for the aggregation
review_metadata = df_AWE[['reviewID', 'Resturant', 'City', 'State', 'Zipcode',	'Business_Rating_Score']].copy()
review_metadata.head()

,reviewID,Resturant,City,State,Zipcode,Business_Rating_Score
0,0,Pizza Hut,Orlando,FL,32819,3.5
1,1,La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0
2,2,Pizza Hut,Orlando,FL,32819,3.5
3,3,Teavana,Atlanta,GA,30326,3.0
4,4,Native Foods Cafe,Happy Valley,OR,97086,4.5


In [ ]:
# Get the AWE vectors for the aggregation
awe_df = df_AWE.iloc[:, -300:].copy()
awe_df.head()

,AWE1,AWE2,AWE3,AWE4,AWE5,AWE6,AWE7,AWE8,AWE9,AWE10,...,AWE291,AWE292,AWE293,AWE294,AWE295,AWE296,AWE297,AWE298,AWE299,AWE300
0,0.000466,0.000839,0.026261,0.006459,-0.023641,-0.001470,-0.003408,-0.079484,-0.033978,-0.007223,...,-0.005968,0.025351,-0.029017,0.000196,0.017939,-0.012416,0.011076,0.014994,0.000625,-0.006430
1,-0.010688,-0.001212,0.020719,0.003003,-0.007375,-0.020803,-0.000588,-0.096937,-0.016457,0.001883,...,0.002474,0.011572,-0.015873,-0.002296,-0.001984,-0.005425,-0.004394,-0.010526,-0.005332,-0.009384
2,-0.005978,-0.000683,0.010486,0.001734,-0.023263,-0.010447,-0.017599,-0.069113,-0.018448,0.014473,...,-0.000389,0.017479,-0.027804,-0.004635,0.007689,-0.005709,0.014056,0.007208,-0.013408,-0.009189
3,-0.006715,0.001068,0.020165,0.007803,-0.042985,-0.004677,-0.007698,-0.067961,-0.017233,0.007675,...,-0.013978,0.025447,-0.008755,-0.001572,0.000808,-0.004360,0.009962,0.019730,-0.025982,0.002044
4,-0.013090,0.010157,0.010822,0.008238,0.001968,-0.008857,-0.010102,-0.070799,-0.021641,-0.004766,...,0.003400,0.015685,-0.023062,0.005128,0.005895,-0.000231,0.004912,-0.014134,-0.007396,-0.004058


In [ ]:
# Put the reviewID for the join
awe_df['reviewID'] = df_AWE['reviewID']
awe_df.head()

,AWE1,AWE2,AWE3,AWE4,AWE5,AWE6,AWE7,AWE8,AWE9,AWE10,...,AWE292,AWE293,AWE294,AWE295,AWE296,AWE297,AWE298,AWE299,AWE300,reviewID
0,0.000466,0.000839,0.026261,0.006459,-0.023641,-0.001470,-0.003408,-0.079484,-0.033978,-0.007223,...,0.025351,-0.029017,0.000196,0.017939,-0.012416,0.011076,0.014994,0.000625,-0.006430,0
1,-0.010688,-0.001212,0.020719,0.003003,-0.007375,-0.020803,-0.000588,-0.096937,-0.016457,0.001883,...,0.011572,-0.015873,-0.002296,-0.001984,-0.005425,-0.004394,-0.010526,-0.005332,-0.009384,1
2,-0.005978,-0.000683,0.010486,0.001734,-0.023263,-0.010447,-0.017599,-0.069113,-0.018448,0.014473,...,0.017479,-0.027804,-0.004635,0.007689,-0.005709,0.014056,0.007208,-0.013408,-0.009189,2
3,-0.006715,0.001068,0.020165,0.007803,-0.042985,-0.004677,-0.007698,-0.067961,-0.017233,0.007675,...,0.025447,-0.008755,-0.001572,0.000808,-0.004360,0.009962,0.019730,-0.025982,0.002044,3
4,-0.013090,0.010157,0.010822,0.008238,0.001968,-0.008857,-0.010102,-0.070799,-0.021641,-0.004766,...,0.015685,-0.023062,0.005128,0.005895,-0.000231,0.004912,-0.014134,-0.007396,-0.004058,4


In [ ]:
# merge join with the original dataset using reviewID 
review_AWE = pd.merge(left=review_metadata, right=awe_df, left_on='reviewID', right_on='reviewID')
review_AWE.head(3)

,reviewID,Resturant,City,State,Zipcode,Business_Rating_Score,AWE1,AWE2,AWE3,AWE4,...,AWE291,AWE292,AWE293,AWE294,AWE295,AWE296,AWE297,AWE298,AWE299,AWE300
0,0,Pizza Hut,Orlando,FL,32819,3.5,0.000466,0.000839,0.026261,0.006459,...,-0.005968,0.025351,-0.029017,0.000196,0.017939,-0.012416,0.011076,0.014994,0.000625,-0.006430
1,1,La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,-0.010688,-0.001212,0.020719,0.003003,...,0.002474,0.011572,-0.015873,-0.002296,-0.001984,-0.005425,-0.004394,-0.010526,-0.005332,-0.009384
2,2,Pizza Hut,Orlando,FL,32819,3.5,-0.005978,-0.000683,0.010486,0.001734,...,-0.000389,0.017479,-0.027804,-0.004635,0.007689,-0.005709,0.014056,0.007208,-0.013408,-0.009189


In [ ]:
# final number of rows in the cleaned dataframe
print("final number of rows: ", len(review_AWE))

final number of rows:  63216


In [ ]:
review_AWE.drop(['reviewID'], axis=1, inplace = True)

In [ ]:
# groupby to with the columns Resturant,	City,	State,	Zipcode,	Business_Rating_Score
# Use group.mean to count the average for each business
review_AWE_gb = review_AWE.groupby(['Resturant', 'Zipcode', 'City', 'State', 'Business_Rating_Score']).mean().reset_index()
print("final number of rows: ", len(review_AWE_gb))

final number of rows:  1075


### Answer to Section 4

Snapshot of the first 5 rows of the aggregated restaurant dataframe with document vectors and original metadata.

In [ ]:
review_AWE_gb.head()

,Resturant,Zipcode,City,State,Business_Rating_Score,AWE1,AWE2,AWE3,AWE4,AWE5,...,AWE291,AWE292,AWE293,AWE294,AWE295,AWE296,AWE297,AWE298,AWE299,AWE300
0,1/2 Roll,V5V 3P6,Vancouver,BC,3.0,-0.004233,0.003970,0.014558,0.007793,-0.013913,...,-0.004434,0.014302,-0.024087,0.002030,0.007161,-0.007361,0.001872,-0.008344,-0.008499,-0.005230
1,21 Nickels,02472,Watertown,MA,4.0,-0.008349,0.002013,0.016399,0.006510,-0.015617,...,0.003085,0.015425,-0.022232,0.002130,0.005787,-0.009329,0.001661,-0.003716,-0.010980,-0.004237
2,24 Hour Pizza Delivery,02124,Boston,MA,1.0,-0.005781,0.005477,0.007118,0.006436,-0.009295,...,0.001349,0.007803,-0.024818,0.003353,0.006289,-0.012131,-0.000909,-0.009797,-0.013009,-0.008563
3,3 Monkeys Pub & Grill,98665,Vancouver,WA,3.0,-0.005030,0.001342,0.017022,0.006513,-0.014102,...,0.002463,0.015094,-0.018789,0.001552,0.004313,-0.007662,0.002451,-0.003775,-0.009389,-0.003387
4,5th St Ribs-N-Blues,30308,Atlanta,GA,3.0,-0.007999,0.004992,0.014365,0.007480,-0.014153,...,0.004341,0.012977,-0.022155,0.004655,0.008265,-0.007713,0.002111,-0.006313,-0.011318,-0.004468


In [ ]:
# Save the dataframe into our drive, so we can load it later for analysis
review_AWE_gb.to_csv('/content/drive/MyDrive/Colab Notebooks/Mydata/restaurant_review_AWE_groupby.csv', index=False)

## Section 5: Sentiment Analysis

### Create Sentiment Analysis Scores for each Individual Review

TextBlob polarity and subjectivity, and Vader compound score.

#### Create a new dataframe with just the columns we need

In [ ]:
# The imports needed for this notebook
import numpy as np
import pandas as pd

In [ ]:
# Automatically mount google drive using the file browser or
# mount google drive using code below
from google.colab import drive
drive.mount('/content/drive',  force_remount=False)

Mounted at /content/drive


In [ ]:
# read a local csv file into Pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Mydata/restaurant_review_row.csv')
df.head(3)

,Unnamed: 0,Cust_Rating,Datetime,Review,Resturant,City,State,Zipcode,Business_Rating_Score,Review_expanded,Review_Token,Review_tagged,Review_lemmas,Review_lemmas_final,Cleaned_review_lemma
0,0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...,"['its', 'the', 'best', 'pizza', 'hut', 'i', ""'...","[('its', 'PRP$'), ('the', 'DT'), ('best', 'JJS...","['its', 'the', 'best', 'pizza', 'hut', 'i', ""'...","['best', 'pizza', 'hut', 'find', 'florida', 'a...",best pizza hut find florida always get great d...
1,1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ...","['This', 'place', 'is', 'an', 'interesting', '...","[('This', 'DT'), ('place', 'NN'), ('is', 'VBZ'...","['This', 'place', 'be', 'an', 'interesting', '...","['place', 'interesting', 'combo', 'chef', 'dav...",place interesting combo chef david incredible ...
2,2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...,"['Pizza', 'Hut', 'is', 'great', '!', 'You', 'g...","[('Pizza', 'NNP'), ('Hut', 'NNP'), ('is', 'VBZ...","['Pizza', 'Hut', 'be', 'great', '!', 'You', 'g...","['pizza', 'hut', 'great', 'get', 'huge', 'pizz...",pizza hut great get huge pizza money service q...


In [ ]:
# Get the AWE vectors for the aggregation
df_SA = df.iloc[:, 4:10].copy()
df_SA.head()

,Resturant,City,State,Zipcode,Business_Rating_Score,Review_expanded
0,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...
1,La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ..."
2,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...
3,Teavana,Atlanta,GA,30326,3.0,Always a great place to taste some tea with fr...
4,Native Foods Cafe,Happy Valley,OR,97086,4.5,Delish! Stopped in on a random Sunday and so g...


In [ ]:
# final number of rows in the cleaned dataframe
print("final number of rows: ", len(df_SA))

final number of rows:  63230


#### Add the polarity and subjectivity measures from TextBlob

In [ ]:
from textblob import TextBlob

In [ ]:
# Add Polarity score
%time df_SA['polarity'] = df_SA['Review_expanded'].apply(lambda row: TextBlob (row).sentiment[0])

df_SA.head(3)

,Resturant,City,State,Zipcode,Business_Rating_Score,Review_expanded,polarity
0,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...,0.527183
1,La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ...",0.190646
2,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...,0.605208


In [ ]:
# Add Subjectivity score
%time df_SA['subjectivity'] = df_SA['Review_expanded'].apply(lambda row: TextBlob (row).sentiment[1])

df_SA.head(3)

CPU times: user 49.3 s, sys: 175 ms, total: 49.4 s
Wall time: 49.5 s


,Resturant,City,State,Zipcode,Business_Rating_Score,Review_expanded,polarity,subjectivity
0,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...,0.527183,0.563492
1,La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ...",0.190646,0.449235
2,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...,0.605208,0.762500


#### Add the compound score from NLTK

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
# add the compound score to the dataframe.

%time df_SA['NLTK_Compound'] = df_SA['Review_expanded'].apply(lambda row: sia.polarity_scores(row)['compound'])
df_SA.head(3)

CPU times: user 1min 18s, sys: 544 ms, total: 1min 18s
Wall time: 1min 19s


,Resturant,City,State,Zipcode,Business_Rating_Score,Review_expanded,polarity,subjectivity,NLTK_Compound
0,Pizza Hut,Orlando,FL,32819,3.5,its the best pizza hut i'v found in florida al...,0.527183,0.563492,0.9712
1,La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"This place is an interesting combo. The chef, ...",0.190646,0.449235,0.9612
2,Pizza Hut,Orlando,FL,32819,3.5,Pizza Hut is great! You get huge pizzas for yo...,0.605208,0.762500,0.9650


### Aggregate by Restaurant

And calculate the mean of the sentiment score.

In [ ]:
%time df_aggre = df_SA.groupby(['Resturant', 'Zipcode', 'City', 'State', 'Business_Rating_Score'], as_index=False).mean()

CPU times: user 36.6 ms, sys: 38 ms, total: 74.6 ms
Wall time: 76 ms


In [ ]:
# final number of rows in the cleaned dataframe
print("final number of rows: ", len(df_aggre))

final number of rows:  1075


### Answer to Section 5 - The first five restaurants and their SA scores

In [ ]:
df_aggre.head()

,Resturant,Zipcode,City,State,Business_Rating_Score,polarity,subjectivity,NLTK_Compound
0,1/2 Roll,V5V 3P6,Vancouver,BC,3.0,0.165462,0.492290,0.471209
1,21 Nickels,02472,Watertown,MA,4.0,0.262287,0.572482,0.798269
2,24 Hour Pizza Delivery,02124,Boston,MA,1.0,-0.131477,0.569455,-0.302206
3,3 Monkeys Pub & Grill,98665,Vancouver,WA,3.0,0.207397,0.569778,0.639382
4,5th St Ribs-N-Blues,30308,Atlanta,GA,3.0,0.228448,0.575966,0.647726


In [ ]:
# Write the final aggregated restraunts and their SA scores to a CSV file.
# Save the dataframe into our drive, so we can load it later for analysis
df_aggre.to_csv('/content/drive/MyDrive/Colab Notebooks/Mydata/restaurant_review_SA_groupby.csv', index=False)